In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Input, BatchNormalization
from tensorflow.keras import Sequential, Model
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

/home/ninja/anaconda3/envs/DS1_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ninja/anaconda3/envs/DS1_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ninja/anaconda3/envs/DS1_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ninja/anaconda3/envs/DS1_env/lib/python3

In [2]:
TRAIN_DATA_PATH = 'data_60_samples300000.npz'
# FWD_INPUT_SHAPE = (5, )
# FWD_OUTPUT_UNITS = 60

# BWD_INPUT_SHAPE = (60, )
# BWD_OUTPUT_UNITS = 5

In [3]:
class TandemModel:
    def __init__(self, path_to_data):
        self.path = path_to_data
        self.x_train = None
        self.y_train = None
        self.x_test = None
        self.y_test = None
        
        # load data
        with np.load(self.path) as data:
            self.x_train, self.x_test, self.y_train, self.y_test = train_test_split(data['X'], data['Y'], test_size=0.1, shuffle=True)
            
        self.fwd_input_shape = self.x_train.shape[1:]
        self.fwd_ouput_units = self.y_train.shape[1]
        
        self.bwd_input_shape = self.y_train.shape[1:]
        self.bwd_output_units = self.x_train.shape[1]
        
        self.fwd = None
        self.bwd = None
        self.tandem = None

        
    def build_fwd_model(self):
        """
        predicts target response based on design parameters
        """
        self.fwd = Sequential(name='fwd_model')
        
        inputs = Input(shape=self.fwd_input_shape, name='fwd_input_layer')
        
        x = Dense(64, activation='relu', name='fwd_hidden_layer1')(inputs)
        x = BatchNormalization()(x)
        x = Dense(64, activation='relu', name='fwd_hidden_layer2')(x)
        x = BatchNormalization()(x)
        x = Dense(64, activation='relu', name='fwd_hidden_layer3')(x)
        x = BatchNormalization()(x)
        x = Dense(64, activation='relu', name='fwd_hidden_layer4')(x)
        x = BatchNormalization()(x)
        
        outputs = Dense(self.fwd_ouput_units, name='fwd_output_layer', activation='sigmoid')(x)
        
        self.fwd = Model(inputs=inputs, outputs=outputs)
        
#         self.fwd.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

    
    def load_fwd_saved_model(self, path):
        self.fwd = load_model(path)
        
    
    def build_bwd_model(self):
        """
        predicts design parameters based on target response
        """
        inputs = Input(shape=self.bwd_input_shape, name='bwd_input_layer')
        
        x = Dense(64, activation='relu', name='bwd_hidden_layer1')(inputs)
        x = BatchNormalization()(x)
        x = Dense(64, activation='relu', name='bwd_hidden_layer2')(x)
        x = BatchNormalization()(x)
        x = Dense(64, activation='relu', name='bwd_hidden_layer3')(x)
        x = BatchNormalization()(x)
        x = Dense(32, activation='relu', name='bwd_hidden_layer4')(x)
        x = BatchNormalization()(x)
        x = Dense(self.bwd_output_units, name='bwd_output_layer', activation='sigmoid')(x)
        
        self.bwd = Model(inputs, x)
    

    def build_tandem_model(self):
        x = self.bwd.output
        
        for layer in self.fwd.layers[1:]:
            layer.trainable = False
            x = layer(x)
            
        self.tandem = Model(inputs=self.bwd.input, outputs=x)
        
    
    def load_bwd_saved_model(self, path):
        self.bwd = load_model(path)
    
    
    def summary(self, model='fwd'):
        if(model == 'fwd'):
            return self.fwd.summary()
        return self.bwd.summary()
    
    
    def fit_fwd(self, batch_size=64, epochs=5, validation_split=0.2):
        return self.fwd.fit(self.x_train, self.y_train, batch_size=batch_size, epochs=epochs, validation_split=0.2, shuffle=True)
        
    
    def save_model(self, path, model='fwd'):
        if model == 'fwd':
            self.fwd.save(path)
        else:
            self.bwd.save(path)
    
    def plot(self, history):
        plt.plot(history.epoch, history.history['accuracy'])
        plt.plot(history.epoch, history.history['loss'])
        plt.xlabel('epochs')
        plt.legend(['accuracy', 'loss'], loc='upper left')
    

In [4]:
tandem = TandemModel(TRAIN_DATA_PATH)

In [5]:
print((tandem.fwd_input_shape, tandem.fwd_ouput_units), (tandem.bwd_input_shape, tandem.bwd_output_units))

((5,), 60) ((60,), 5)


In [6]:
tandem.load_fwd_saved_model('fwd_64x64x64x64x60_50epochs300000samples_56%_adam')
# tandem.build_fwd_model()
tandem.build_bwd_model()
tandem.build_tandem_model()
tandem.bwd.summary()
tandem.fwd.summary()
tandem.tandem.summary()
tandem.tandem.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bwd_input_layer (InputLayer) [(None, 60)]              0         
_________________________________________________________________
bwd_hidden_layer1 (Dense)    (None, 64)                3904      
_________________________________________________________________
batch_normalization (BatchNo (None, 64)                256       
_________________________________________________________________
bwd_hidden_layer2 (Dense)    (None, 64)                4160      
_________________________________________________________________
batch_normalization_1 (Batch (None, 64)                256       
_________________________________________________________________
bwd_hidden_layer3 (Dense)    (None, 64)                4160      
_________

In [11]:
history = tandem.tandem.fit(tandem.y_train, tandem.y_train, epochs=50, batch_size=128, validation_split=0.2, shuffle=True)

Train on 216000 samples, validate on 54000 samples
Epoch 1/50
216000/216000 [==============================] - 8s 35us/sample - loss: 0.0042 - acc: 0.4306 - val_loss: 0.0029 - val_acc: 0.4789
Epoch 2/50
216000/216000 [==============================] - 8s 36us/sample - loss: 0.0039 - acc: 0.4416 - val_loss: 0.0029 - val_acc: 0.4706
Epoch 3/50
216000/216000 [==============================] - 7s 35us/sample - loss: 0.0038 - acc: 0.4434 - val_loss: 0.0031 - val_acc: 0.4653
Epoch 4/50
216000/216000 [==============================] - 8s 36us/sample - loss: 0.0038 - acc: 0.4441 - val_loss: 0.0037 - val_acc: 0.4868
Epoch 5/50
216000/216000 [==============================] - 8s 36us/sample - loss: 0.0037 - acc: 0.4499 - val_loss: 0.0024 - val_acc: 0.5055
Epoch 6/50
216000/216000 [==============================] - 8s 36us/sample - loss: 0.0035 - acc: 0.4577 - val_loss: 0.0028 - val_acc: 0.4580
Epoch 7/50
216000/216000 [==============================] - 8s 36us/sample - loss: 0.0035 - acc: 0.4535

In [13]:
tandem.tandem.evaluate(tandem.y_test, tandem.y_test)

30000/30000 [==============================] - 1s 36us/sample - loss: 0.0020 - acc: 0.5165


[0.0019933224784210325, 0.5164667]

In [105]:
keras.utils.plot_model(tandem.tandem)

Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.


In [12]:
tandem.bwd.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bwd_input_layer (InputLayer) [(None, 60)]              0         
_________________________________________________________________
bwd_hidden_layer1 (Dense)    (None, 64)                3904      
_________________________________________________________________
batch_normalization (BatchNo (None, 64)                256       
_________________________________________________________________
bwd_hidden_layer2 (Dense)    (None, 64)                4160      
_________________________________________________________________
batch_normalization_1 (Batch (None, 64)                256       
_________________________________________________________________
bwd_hidden_layer3 (Dense)    (None, 64)                4160      
_________________________________________________________________
batch_normalization_2 (Batch (None, 64)                256   

In [14]:
tandem.bwd.save('bwd_64x64x64x32x5_adam_50epochs_51%_300000samples')

In [25]:
with np.load(TRAIN_DATA_PATH) as data:
    X = data['X']
    Y = data['Y']

In [ ]:
print(X.shape, Y.shape)

In [ ]:
y_train, y_test, x_train, x_test = train_test_split(X, Y, test_size=0.1, shuffle=True)

In [ ]:
# clear memory size ~400MB
del X
del Y

In [ ]:
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)
print(y_test.shape[1:])

In [2]:
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential()
model.add(layers.Input(shape=(60, )))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# tandem.fwd.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
tandem.fwd.trainable = False
# tandem.fit_fwd(batch_size=128, epochs=1)
tandem.summary()

In [ ]:
model = build_model()

In [ ]:
model.summary()

In [ ]:
epochs = 50
batch_size = 64

In [ ]:
%time
history = model.fit(x_train, y_train, batch_size=64, epochs=epochs, validation_split=0.2, shuffle=True)

In [ ]:
xc = range(5)
plt.plot(history.history['accuracy'])
plt.plot(history.history['loss'])
plt.xlabel('epochs')
plt.legend(['accuracy', 'loss'], loc='upper left')

In [ ]:
test_scores = model.evaluate(x_test, y_test, verbose=2)
print("Test loss:", test_scores[0])
print("Test accuracy:", test_scores[1])

In [ ]:
predictions = model.predict(x_test)
for i in range(150, 170):
    fig, ax = plt.subplots()
    ax.plot(y_test[i])
    ax.plot(predictions[i])
    ax.legend(['actual', 'predicted'])

In [ ]:
model.save('fwd_64x64x64x64x60_50epochs_56%_adam')